<a href="https://colab.research.google.com/github/riverdogcabin/PSDS4900/blob/main/wu_observation_modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from pandas.api.types import is_numeric_dtype
import numpy as np
import scipy.stats as stats
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold
import datetime
import matplotlib.pyplot as plt
from matplotlib.dates import DayLocator, DateFormatter
from pprint import pprint
import json

In [2]:
with open('/content/drive/MyDrive/Capstone/PSDS4900/config.json') as configuration:
  my_station = json.load(configuration).get('WU')["stationid"] #'WU' is the parameters for WeatherUnderground

In [3]:
df = pd.read_csv('/content/drive/MyDrive/Capstone/PSDS4900/wu_data/wu_observations.csv')
df['timestamp'] = pd.to_datetime(df['epoch'], unit='s')
df = df.assign(timestamp_rounded=df.timestamp.dt.round('60min')) #rounded to the hour
print('with dupes',df.shape)
before = df.shape[0]
df.drop_duplicates(inplace=True)
print('without dupes',df.shape)
print('difference: ',before-df.shape[0])

with dupes (19504, 34)
without dupes (19504, 34)
difference:  0


### Clean up the spikes (dupes in each timegroup as outline din the stats notebook)

In [4]:
num_stations = len(df.stationID.unique())
counted_df = df.groupby(['timestamp_rounded']).count().reset_index()
hour_grouping_std = counted_df.stationID.std()
problem_hours = list(counted_df[counted_df.stationID > num_stations+hour_grouping_std].timestamp_rounded) #get all groups that are more than one standard deviation off the station count
indices_to_drop = df[(df.timestamp_rounded.isin(problem_hours)) & (df.timestamp > df.timestamp_rounded)].index
df.drop(indices_to_drop,inplace=True)

In [5]:
# uncomment the below to check to make sure the spikes are gone
# grouped = df.groupby(['timestamp_rounded'])
# ax = grouped.count().reset_index().windspeedAvg.plot(xlabel='Group Number',ylabel='Count',figsize=(20, 10))
# ax.axhline(y=55,color='green',linestyle='--')
# ax.axhline(y=num_stations,color='red',linestyle='--')

### Build correlation DataFrame and the dict of maximally correlated stations

In [6]:
columns_to_check = [s for s in df.columns if "Avg" in s]
corr_df = pd.DataFrame({'stationID':df.stationID.unique()}) #create the shell of the dataframe to store the correlations
max_correlations = {} #this will hold the maximally correlated stations for each variable
for v in columns_to_check:
  #create a pivot table for each variable
  temp_df = df.pivot_table(index='timestamp_rounded',columns='stationID',values=v).corr()[[my_station]]
  temp_df.columns.name = None #get rid of the column names and then collapse the indices, reindex and rename the columns
  temp_df = temp_df.stack().reset_index().drop(columns='level_1').rename(columns={0:v+'_corr'})
  #get rid of the results for my_station, obvs 
  temp_df = temp_df[temp_df.stationID != my_station]
  #collect the maximally correlated station for the variable
  max_correlations[v] = temp_df.loc[temp_df[v+'_corr'].abs().idxmax()]
  #add all the corrleations for this variable as a column to the big correlation dataframe
  corr_df = corr_df.merge(temp_df,on='stationID')

### Now let's build a simple linear regression model and test it. Found a[ great code snippet to set up k-fold cross validation](https://becominghuman.ai/linear-regression-in-python-with-pandas-scikit-learn-72574a2ec1a5) and adapted it. We'll start by just using the most highly-correlated station to create the model.

In [7]:
target_stations = [my_station, max_correlations['windspeedAvg'].stationID]
df_filtered = df[df.stationID.isin(target_stations)]
df_reshaped = df_filtered.pivot(index='timestamp_rounded',columns='stationID',values='windspeedAvg')
# df_reshaped

In [8]:
X = pd.DataFrame(df_reshaped.drop(columns=my_station))
y = pd.DataFrame(df_reshaped[[my_station]]) #target

model = LinearRegression()
scores = []
kfold = KFold(n_splits=3, shuffle=True, random_state=42)

for i, (train, test) in enumerate(kfold.split(X, y)):
 model.fit(X.iloc[train,:], y.iloc[train,:])
 score = model.score(X.iloc[test,:], y.iloc[test,:])
 scores.append(score)
print(scores)

[0.585806568136984, 0.6961795230891136, 0.7735785348303749]


### Those scores aren't bad, but they aren't great. Let's see if adding more stations improves things. We'll take the top five most correlated stations.

In [9]:
target_stations = corr_df.sort_values('windspeedAvg_corr').stationID.tail().to_list()
target_stations.append(my_station)
df_filtered = df[df.stationID.isin(target_stations)]
df_reshaped = df_filtered.pivot_table(index='timestamp_rounded',columns='stationID',values='windspeedAvg')
before = df_reshaped.shape[0]
df_reshaped.dropna(inplace=True)
print("dropping N/A rows reduced from {} to {}. Loss of {} rows".format(before,df_reshaped.shape[0],before - df_reshaped.shape[0]))
# pd.options.display.max_rows = 300 #so we can show the whole dataframe
# df_reshaped

dropping N/A rows reduced from 263 to 163. Loss of 100 rows


In [10]:
X = pd.DataFrame(df_reshaped.drop(columns=my_station))
y = pd.DataFrame(df_reshaped[[my_station]]) #target

model = LinearRegression()
scores = []
kfold = KFold(n_splits=3, shuffle=True, random_state=42)

for i, (train, test) in enumerate(kfold.split(X, y)):
 model.fit(X.iloc[train,:], y.iloc[train,:])
 score = model.score(X.iloc[test,:], y.iloc[test,:])
 scores.append(score)
print(scores)

[0.9010541746729965, 0.7270595151373567, 0.8617657323616549]


### Those got better by adding four more highly-correlated stations. Let's see what happens when we add all the stations.

In [11]:
target_stations = corr_df.sort_values('windspeedAvg_corr').stationID.to_list()
target_stations.append(my_station)
df_filtered = df[df.stationID.isin(target_stations)]
df_reshaped = df_filtered.pivot_table(index='timestamp_rounded',columns='stationID',values='windspeedAvg')
before = df_reshaped.shape[0]
df_reshaped.dropna(inplace=True)
print("dropping N/A rows reduced from {} to {}. Loss of {} rows".format(before,df_reshaped.shape[0],before - df_reshaped.shape[0]))

dropping N/A rows reduced from 263 to 73. Loss of 190 rows


In [12]:
X = pd.DataFrame(df_reshaped.drop(columns=my_station))
y = pd.DataFrame(df_reshaped[[my_station]]) #target

model = LinearRegression()
scores = []
kfold = KFold(n_splits=3, shuffle=True, random_state=42)

for i, (train, test) in enumerate(kfold.split(X, y)):
 model.fit(X.iloc[train,:], y.iloc[train,:])
 score = model.score(X.iloc[test,:], y.iloc[test,:])
 scores.append(score)
print(scores)

[0.46941616151915594, 0.27298305173122706, 0.7163418274249742]


### That did *not* improve things. I think that we had to drop too many rows because of missing values. Even though the hourly grouping was the _best_ way to align the data, it still means we lose lots of rows because of missing values. 

### The next few cells are a quick aside to figure out which stationIDs and and for what dates we are missing so much data. This will be used in another script to pull those stations/dates but the data exists here so it saves time to do it here. 

In [16]:
target_stations = corr_df.sort_values('windspeedAvg_corr').stationID.to_list()
target_stations.append(my_station)
df_filtered = df[df.stationID.isin(target_stations)]
df_reshaped = df_filtered.pivot_table(index='timestamp_rounded',columns='stationID',values='windspeedAvg')

In [14]:
temp_df = df_reshaped.loc['2021-03-16 07:00:00'].T
temp_df[temp_df.isna()].index.to_list()

['KCOCASTL167',
 'KCOCASTL26',
 'KCOCASTL308',
 'KCOCASTL312',
 'KCOCASTL321',
 'KCOCASTL331',
 'KCOCASTL342',
 'KCOCASTL390',
 'KCOCASTL397',
 'KCOCASTL405',
 'KCOCASTL430',
 'KCOLITTL212',
 'KCOLITTL697',
 'KCOLITTL720',
 'KCOLITTL771',
 'KCOLITTL82',
 'KCOLOUVI2',
 'KCOSEDAL14',
 'KCOSEDAL28',
 'KCOSEDAL29',
 'KCOSEDAL4',
 'KCOSEDAL42',
 'KCOSEDAL52',
 'KCOSEDAL54',
 'KCOSEDAL6',
 'KCOSEDAL61',
 'KCOSEDAL71',
 'KCOSEDAL72',
 'KCOSEDAL73',
 'KCOSILVE32']

In [15]:
temp_df = df_reshaped.T.loc['KCOCASTL167']
# temp_df.head()
date_set = set()
for d in temp_df[temp_df.isna()].index.to_list():
  date_set.add('{}{}{}'.format(d.year,d.month,d.day))

date_set

{'2021316', '2021317', '2021318', '2021319', '2021320'}